# Hardware
### DGX Station
V100 SMX2 16GB

Intel Xeon E5-2968 v4 @ 2.20GHz

# Software
CUDA 11.0

cuSignal 0.19

SciPy 1.6.3

CuPy 9.0.0

# Profiling
[Nsight Systems](https://docs.nvidia.com/nsight-systems/UserGuide/index.html)

## CPU Baseline
Baseline Scipy's Signal Lombscargle function

In [1]:
!nsys profile -s none -o scipy_v1 -f true --stats=true python3 icassp_scipy_v1.py float32 5

Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-76d9-cd22-0527-57d3.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-76d9-cd22-0527-57d3.qdrep"
Exporting 643307 events: [================================================100%]

Exported successfully to
/tmp/nsys-report-76d9-cd22-0527-57d3.sqlite

Generating CUDA API Statistics...
CUDA API Statistics (nanoseconds)




CUDA trace data was not collected.


Generating Operating System Runtime API Statistics...
Operating System Runtime API Statistics (nanoseconds)

Time(%)      Total Time       Calls         Average         Minimum         Maximum  Name                                                                            
-------  --------------  ----------  --------------  --------------  --------------  ----------------------------------------------------------------------------

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 51,075.2 | 51,128.2    | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 50.799.1 | 50,806.4    | 1.0      |
| CuPy (Baseline)       |           |          |             |          |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

## GPU Scenario #1
1. CuPy implementation of Lombscargle function

In [2]:
!nsys profile -s none -o cupy_v1 -f true --stats=true python3 icassp_cupy_v1.py float32 2000 0

Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-caf7-5a97-57fb-5eb1.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-caf7-5a97-57fb-5eb1.qdrep"
Exporting 653033 events: [================================================100%]

Exported successfully to
/tmp/nsys-report-caf7-5a97-57fb-5eb1.sqlite

Generating CUDA API Statistics...
CUDA API Statistics (nanoseconds)

Time(%)      Total Time       Calls         Average         Minimum         Maximum  Name                                                                            
-------  --------------  ----------  --------------  --------------  --------------  --------------------------------------------------------------------------------
   97.8     20056687899        2001      10023332.3         9562873        10851736  cudaDeviceSynchronize                                      

In [3]:
#!nsys stats --report gpukernsum --report nvtxppsum --report gputrace cupy_v1.qdrep # Use to get register usage
!nsys stats --report gpukernsum --report nvtxppsum cupy_v1.qdrep

Using cupy_v1.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v1.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average      Minimum    Maximum          Name       
 -------  ---------------  ---------  ------------  ---------  ----------  -----------------
   100.0   20,053,721,658      2,001  10,021,849.9  9,562,272  10,849,989  _cupy_lombscargle

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v1.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum            Range        
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    99.3   20,244,660,806      2,000   10,122,330.4    9,662,292   10,950,094  cupy_lombscargle_loop
     0.7      137,536,440          1  137,536,440.0  137,536,440  137,536,440  cupy_lombscargle     



### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 51,075.2 | 51,128.2    | 1.0      |
| CuPy (Baseline)       | 30        | 135.1    | 10.1        | 5062.2   |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 50.799.1 | 50,806.4    | 1.0      |
| CuPy (Baseline)       | 48        | 145.0    | 21.2        | 2396.5   |
| CuPy (User Cache)     |           |          |             |          |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

## GPU Scenario #2
1. CuPy implementation of Lombscargle function
2. Added user cache

In [4]:
!nsys profile -s none -o cupy_v2 -f true python3 icassp_cupy_v2.py float32 2000 0

Registers 30
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-135b-ce84-6218-c1e1.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-135b-ce84-6218-c1e1.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v2.qdrep"


In [5]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v2.qdrep

Generate SQLite file cupy_v2.sqlite from cupy_v2.qdrep
Exporting 635440 events: [================================================100%]
Using cupy_v2.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v2.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average      Minimum    Maximum          Name       
 -------  ---------------  ---------  ------------  ---------  ----------  -----------------
   100.0   20,070,227,792      2,001  10,030,098.8  9,565,922  11,214,888  _cupy_lombscargle

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v2.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum            Range        
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    99.3   20,177,914,899      2,000   10,088,957.4    9,623,261   11,272,907  cupy_lombscargle_loop
    

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 51,075.2 | 51,128.2    | 1.0      |
| CuPy (Baseline)       | 30        | 135.1    | 10.1        | 5062.2   |
| CuPy (User Cache)     | 30        | 137.2    | 10.1        | 5062.2   |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 50.799.1 | 50,806.4    | 1.0      |
| CuPy (Baseline)       | 48        | 145.0    | 21.2        | 2396.5   |
| CuPy (User Cache)     | 48        | 145.7    | 21.2        | 2396.5   |
| CuPy (Data Type)      |           |          |             |          |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

## GPU Scenario #3
1. CuPy implementation of Lombscargle function
2. Added user cache
3. Specific functions per data type

In [6]:
!nsys profile -s none -o cupy_v3 -f true python3 icassp_cupy_v3.py float32 2000 0

Registers 26
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-fbcb-de55-99a3-8d1c.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-fbcb-de55-99a3-8d1c.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v3.qdrep"


In [7]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v3.qdrep

Generate SQLite file cupy_v3.sqlite from cupy_v3.qdrep
Exporting 648951 events: [================================================100%]
Using cupy_v3.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v3.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average      Minimum    Maximum              Name           
 -------  ---------------  ---------  ------------  ---------  ----------  -------------------------
   100.0   20,377,888,995      2,001  10,183,852.6  9,708,131  10,887,367  _cupy_lombscargle_float32

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v3.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum            Range        
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    99.3   20,578,059,624      2,000   10,289,029.8    9,805,129   11,003,895  cu

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 51,075.2 | 51,128.2    | 1.0      |
| CuPy (Baseline)       | 30        | 135.1    | 10.1        | 5062.2   |
| CuPy (User Cache)     | 30        | 137.2    | 10.1        | 5062.2   |
| CuPy (Data Type)      | 26        | 135.4    | 10.3        | 4963.9   |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 50.799.1 | 50,806.4    | 1.0      |
| CuPy (Baseline)       | 48        | 145.0    | 21.2        | 2396.5   |
| CuPy (User Cache)     | 48        | 145.7    | 21.2        | 2396.5   |
| CuPy (Data Type)      | 48        | 150.4    | 21.2        | 2396.5   |
| CuPy (Fast Math)      |           |          |             |          |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

## GPU Scenario #4
1. CuPy implementation of Lombscargle function
2. Added user cache
3. Specific functions per data type
4. Utilize fast_math compiler flag

In [9]:
!nsys profile -s none -o cupy_v4 -f true python3 icassp_cupy_v4.py float32 2000 0

Registers 32
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-59d9-9902-2366-fb3c.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-59d9-9902-2366-fb3c.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v4.qdrep"


In [10]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v4.qdrep

Generate SQLite file cupy_v4.sqlite from cupy_v4.qdrep
Exporting 609810 events: [================================================100%]
Using cupy_v4.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v4.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average     Minimum    Maximum             Name           
 -------  ---------------  ---------  -----------  ---------  ---------  -------------------------
   100.0    4,474,465,748      2,001  2,236,114.8  2,133,767  2,320,873  _cupy_lombscargle_float32

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v4.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances     Average       Minimum      Maximum            Range        
 -------  ---------------  ---------  -------------  -----------  -----------  ---------------------
    97.4    4,658,597,871      2,000    2,329,298.9    2,226,509    2,634,089  cupy_lom

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 51,075.2 | 51,128.2    | 1.0      |
| CuPy (Baseline)       | 30        | 135.1    | 10.1        | 5062.2   |
| CuPy (User Cache)     | 30        | 137.2    | 10.1        | 5062.2   |
| CuPy (Data Type)      | 26        | 135.4    | 10.3        | 4963.9   |
| CuPy (Fast Math)      | 32        | 126.1    | 2.3         | 22229.7  |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 50.799.1 | 50,806.4    | 1.0      |
| CuPy (Baseline)       | 48        | 145.0    | 21.2        | 2396.5   |
| CuPy (User Cache)     | 48        | 145.7    | 21.2        | 2396.5   |
| CuPy (Data Type)      | 48        | 150.4    | 21.2        | 2396.5   |
| CuPy (Fast Math)      | 48        | 146.1    | 21.2        | 2396.5   |
| CuPy (Fatbin)         |           |          |             |          |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

## GPU Scenario #5
1. CuPy implementation of Lombscargle function
2. Added user cache
3. Specific functions per data type
4. Utilize fast_math compiler flag
5. Precompile CUDA kernels into fatbins to be loaded at runtime

In [11]:
!nvcc --fatbin -std=c++11 --use_fast_math \
    --generate-code arch=compute_70,code=sm_70 \
    --generate-code arch=compute_75,code=sm_75 \
    --generate-code arch=compute_80,code=[sm_80,compute_80] \
    _lombscargle.cu -odir .

In [12]:
!nsys profile -s none -o cupy_v5 -f true python3 icassp_cupy_v5.py float32 2000 0

Registers 32
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-106c-2be3-6e40-4ecf.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-106c-2be3-6e40-4ecf.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v5.qdrep"


In [13]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v5.qdrep

Generate SQLite file cupy_v5.sqlite from cupy_v5.qdrep
Exporting 622705 events: [================================================100%]
Using cupy_v5.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v5.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average     Minimum    Maximum             Name           
 -------  ---------------  ---------  -----------  ---------  ---------  -------------------------
   100.0    4,493,516,577      2,001  2,245,635.5  2,137,543  2,597,897  _cupy_lombscargle_float32

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v5.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average     Minimum    Maximum           Range        
 -------  ---------------  ---------  -----------  ---------  ---------  ---------------------
    99.9    4,681,463,056      2,000  2,340,731.5  2,231,473  2,696,755  cupy_lombscargle_loop
    

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 51,075.2 | 51,128.2    | 1.0      |
| CuPy (Baseline)       | 30        | 135.1    | 10.1        | 5062.2   |
| CuPy (User Cache)     | 30        | 137.2    | 10.1        | 5062.2   |
| CuPy (Data Type)      | 26        | 135.4    | 10.3        | 4963.9   |
| CuPy (Fast Math)      | 32        | 126.1    | 2.3         | 22229.7  |
| CuPy (Fatbin)         | 32        | 3.9      | 2.3         | 22229.7  |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 50.799.1 | 50,806.4    | 1.0      |
| CuPy (Baseline)       | 48        | 145.0    | 21.2        | 2396.5   |
| CuPy (User Cache)     | 48        | 145.7    | 21.2        | 2396.5   |
| CuPy (Data Type)      | 48        | 150.4    | 21.2        | 2396.5   |
| CuPy (Fast Math)      | 48        | 146.1    | 21.2        | 2396.5   |
| CuPy (Fatbin)         | 48        | 22.3     | 21.2        | 2396.5   |
| CuPy (Launch Bounds)  |           |          |             |          |
| CuPy (Trig Functions) |           |          |             |          |

## GPU Scenario #6
1. CuPy implementation of Lombscargle function
2. Added user cache
3. Specific functions per data type
4. Utilize fast_math compiler flag
5. Precompile CUDA kernels into fatbins to be loaded at runtime
6. Add launch bounds to CUDA kernels

In [14]:
!nvcc --fatbin -std=c++11 --use_fast_math \
    --generate-code arch=compute_70,code=sm_70 \
    --generate-code arch=compute_75,code=sm_75 \
    --generate-code arch=compute_80,code=[sm_80,compute_80] \
    _lombscargle_lb.cu -odir .

In [15]:
!nsys profile -s none -o cupy_v6 -f true python3 icassp_cupy_v6.py float32 2000 0

Registers 31
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-21c0-8197-8b3d-5e67.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-21c0-8197-8b3d-5e67.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v6.qdrep"


In [16]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v6.qdrep

Generate SQLite file cupy_v6.sqlite from cupy_v6.qdrep
Exporting 635144 events: [================================================100%]
Using cupy_v6.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v6.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average     Minimum    Maximum             Name           
 -------  ---------------  ---------  -----------  ---------  ---------  -------------------------
   100.0    4,481,171,056      2,001  2,239,465.8  2,143,303  2,488,841  _cupy_lombscargle_float32

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v6.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average     Minimum    Maximum           Range        
 -------  ---------------  ---------  -----------  ---------  ---------  ---------------------
    99.9    4,666,106,676      2,000  2,333,053.3  2,236,028  2,626,096  cupy_lombscargle_loop
    

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 51,075.2 | 51,128.2    | 1.0      |
| CuPy (Baseline)       | 30        | 135.1    | 10.1        | 5062.2   |
| CuPy (User Cache)     | 30        | 137.2    | 10.1        | 5062.2   |
| CuPy (Data Type)      | 26        | 135.4    | 10.3        | 4963.9   |
| CuPy (Fast Math)      | 32        | 126.1    | 2.3         | 22229.7  |
| CuPy (Fatbin)         | 32        | 3.9      | 2.3         | 22229.7  |
| CuPy (Launch Bounds)  | 31        | 3.9      | 2.3         | 22229.7  |
| CuPy (Trig Functions) |           |          |             |          |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 50.799.1 | 50,806.4    | 1.0      |
| CuPy (Baseline)       | 48        | 145.0    | 21.2        | 2396.5   |
| CuPy (User Cache)     | 48        | 145.7    | 21.2        | 2396.5   |
| CuPy (Data Type)      | 48        | 150.4    | 21.2        | 2396.5   |
| CuPy (Fast Math)      | 48        | 146.1    | 21.2        | 2396.5   |
| CuPy (Fatbin)         | 48        | 22.3     | 21.2        | 2396.5   |
| CuPy (Launch Bounds)  | 48        | 22.8     | 21.2        | 2396.5   |
| CuPy (Trig Functions) |           |          |             |          |

## GPU Scenario #7
1. CuPy implementation of Lombscargle function
2. Added user cache
3. Specific functions per data type
4. Utilize fast_math compiler flag
5. Precompile CUDA kernels into fatbins to be loaded at runtime
6. Add launch bounds to CUDA kernels
7. Use fast sin + cos functions

In [17]:
!nvcc --fatbin -std=c++11 --use_fast_math \
    --generate-code arch=compute_70,code=sm_70 \
    --generate-code arch=compute_75,code=sm_75 \
    --generate-code arch=compute_80,code=[sm_80,compute_80] \
    _lombscargle_lb_trig.cu -odir .

In [18]:
!nsys profile -s none -o cupy_v7 -f true python3 icassp_cupy_v7.py float32 2000 0

Registers 31
Processing events...
Capturing symbol files...
Saving temporary "/tmp/nsys-report-b5c1-e8bd-fcf4-88f7.qdstrm" file to disk...
Creating final output files...

Processing [==============================================================100%]
Saved report file to "/tmp/nsys-report-b5c1-e8bd-fcf4-88f7.qdrep"
Report file moved to "/home/odysseus/workStuff/cusignal-icassp-tutorial/notebooks/raw_cupy_cuda/cupy_v7.qdrep"


In [19]:
!nsys stats --report gpukernsum --report nvtxppsum cupy_v7.qdrep

Generate SQLite file cupy_v7.sqlite from cupy_v7.qdrep
Exporting 615887 events: [================================================100%]
Using cupy_v7.sqlite file for stats and reports.
Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/gpukernsum cupy_v7.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average     Minimum    Maximum             Name           
 -------  ---------------  ---------  -----------  ---------  ---------  -------------------------
   100.0    4,474,814,029      2,001  2,236,288.9  2,139,815  2,490,057  _cupy_lombscargle_float32

Exporting [/opt/nvidia/nsight-systems/2020.3.2/target-linux-x64/reports/nvtxppsum cupy_v7.sqlite] to console... 

 Time(%)  Total Time (ns)  Instances    Average     Minimum    Maximum           Range        
 -------  ---------------  ---------  -----------  ---------  ---------  ---------------------
    99.9    4,652,614,783      2,000  2,326,307.4  2,226,740  2,602,141  cupy_lombscargle_loop
    

### Single Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 51,075.2 | 51,128.2    | 1.0      |
| CuPy (Baseline)       | 30        | 135.1    | 10.1        | 5062.2   |
| CuPy (User Cache)     | 30        | 137.2    | 10.1        | 5062.2   |
| CuPy (Data Type)      | 26        | 135.4    | 10.3        | 4963.9   |
| CuPy (Fast Math)      | 32        | 126.1    | 2.3         | 22229.7  |
| CuPy (Fatbin)         | 32        | 3.9      | 2.3         | 22229.7  |
| CuPy (Launch Bounds)  | 31        | 3.9      | 2.3         | 22229.7  |
| CuPy (Trig Functions) | 31        | 3.9      | 2.3         | 22229.7  |

### Double Precision
|                       | Registers | JIT (ms) | Kernel (ms) | Speed Up |
|:----------------------|:----------|:---------|:------------|:---------|
| Scipy                 | -         | 50.799.1 | 50,806.4    | 1.0      |
| CuPy (Baseline)       | 48        | 145.0    | 21.2        | 2396.5   |
| CuPy (User Cache)     | 48        | 145.7    | 21.2        | 2396.5   |
| CuPy (Data Type)      | 48        | 150.4    | 21.2        | 2396.5   |
| CuPy (Fast Math)      | 48        | 146.1    | 21.2        | 2396.5   |
| CuPy (Fatbin)         | 48        | 22.3     | 21.2        | 2396.5   |
| CuPy (Launch Bounds)  | 48        | 22.8     | 21.2        | 2396.5   |
| CuPy (Trig Functions) | 48        | 15.5     | 14.1        | 3603.3   |